# 3. Exploratory Data Analysis (EDA)

### Thiết lập ban đầu

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import seaborn as sns

pd.options.mode.copy_on_write = True

In [ ]:
# # For Google Colab
# from google.colab import drive
# from pydrive2.auth import GoogleAuth
# from pydrive2.drive import GoogleDrive
# from google.colab import auth
# from oauth2client.client import GoogleCredentials

# # Auth Google Drive
# auth.authenticate_user()
# gauth = GoogleAuth()
# gauth.credentials = GoogleCredentials.get_application_default()
# drive = GoogleDrive(gauth)

# # Download files on Shared folder to Google Colab

# # Shared folder ID
# folder_id = "1cv2YpR9aqRwocc8jwHLcFopfV9G0S-vH"

# # File name
# needed_files = [
#     "chotot_data.csv",
#     "input_scoli_2023.json",
#     "model_ref_price_extra.csv",
#     "model_ref_price_full.csv",
#     "origin_country_multiplier.csv",
# ]

# # Search for the file in the shared folder
# file_list = drive.ListFile({'q': f"'{folder_id}' in parents"}).GetList()
# file_list_simplified = []
# if file_list:
#     for file in file_list:
#       file_name = file['title']
#       file_id = file['id']
#       if file_name in needed_files:
#         file_list_simplified.append({
#             file_name: file_id
#         })

#         # Download the file to Colab
#         downloaded = drive.CreateFile({'id': file_id})
#         downloaded.GetContentFile(file_name)

# MODEL_REF_PRICE_PATH = "model_ref_price_full.csv"
# MODEL_REF_EXTRA_PRICE_PATH = "model_ref_price_extra.csv"
# COUNTRY_MULTIPLIER_PATH = "origin_country_multiplier.csv"
# SCOLI_PATH = "input_scoli_2023.json"
# INPUT_FILE_PATH = "chotot_data.csv"


# For VSCode
MODEL_REF_PRICE_PATH = "../data/raw/model_ref_price_full.csv"
MODEL_REF_EXTRA_PRICE_PATH = "../data/raw/model_ref_price_extra.csv"
COUNTRY_MULTIPLIER_PATH = "../data/raw/origin_country_multiplier.csv"
SCOLI_PATH = "../data/raw/input_scoli_2023.json"
INPUT_FILE_PATH = "../data/raw/chotot_data.csv"

Xem trước dữ liệu

1. Làm sạch cột
- Giá
- Địa điểm
- Xuất xứ: Lọc thêm thông tin 
- Năm đăng ký

2. Tạo đặc trưng mới (tuổi)

3. Phân tích các biến
- Biến mục tiêu price:
    - Vấn đề về giá trị: Histogram > Sắp xếp > Filter
    - Vấn đề về phân phối: lệch phải > có thể cần biến đổi log
- Biến dự đoán
    - Biến định lượng
        - age: histogram, scatterplot > age_log: histogram, scatterplot
        - mileage: filter > mileage_log
    - Biến định tính:
        - Chuyển đổi sang dạng số: cần biến đổi sang dạng số để sử dụng trong mô hình. Thực hiện thông qua các phương pháp encoding (one-hot encoding, ordinal encoding, ...). Chuyển đổi thành con số thể hiện thứ tự của các giá trị trong biến. Con số nào gắn với mẫu xe, thể hiện mẫu xe này tốt hơn mẫu xe khác, ảnh hưởng đến giá bán xe cũ? Chính là giá bán lẻ trên thị trường. > Tìm các con số tương tự thể hiện thứ tự sắp xếp của các giá trị trong biến định tính.
            - model: giá bán lẻ trên thị trường (model_ref_price)
            - origin: bội số cho biết tầm ảnh hưởng của nước sản xuất đến giá bán. VD: xe Nhật giá thường cao hơn xe Trung. (origin_multiplier)
            - location: chỉ số giá sinh hoạt, cho biết giá cả ở 1 tỉnh này đắt đỏ như nào so với tỉnh khác. (province_scoli)
        - Lấy dữ liệu
            - model_ref_price: crawl
            - origin_multiplier: tự tạo
            - province_scoli: tải từ tổng cục thống kê
        - Biến đổi và kiểm tra quan hệ tuyến tính

4. Tạo mô hình
4.1. Các loại mô hình
4.2. Mô hình hồi quy
4.3. Mô hình Random Forest
4.4. So sánh 2 mô hình